# [**시계열 알고리즘 종류**](https://paperswithcode.com/area/time-series)

**"통계추론, 기계학습 및 딥러닝의 흐름에 `시간패턴`을 반영하려 진화"**

**"`지도학습(예측 분류), 비지도학습` 문제에 모두 활용되는 필수 알고리즘"** 

**"`미래 예측을 포함한 추천 서비스와 같은 비즈니스`에 활용중"** 

<img src='./img/TS_Evolution.png' width='900'>

#### 1) 통계추론(Statistical Inference) 알고리즘

- `통계분포`에 기반한 `설명력 중시 알고리즘`

**(1) 단변량 선형기반:** `Y가 1개` & `Y와 X의 관계를 선형 가정`

- Linear Regression

- `ARIMA(AutoRegressive Integrated Moving Average)`

- `ARIMAX`

- `SARIMAX`

**(2) 다변량 선형기반:** `Y가 2개 이상` & `Y와 X의 관계를 선형 가정`

- Bayesian-based Models

- [`Vector Autoregression(VAR)`](https://en.wikipedia.org/wiki/Vector_autoregression)

- `Vector Error Correction Model(VECM)`

**(3) 비선형기반:** `Y와 X의 관계를 비선형 가정`

- `Exponential Smoothing`

- `ETS(Error/Trend/Seasonal)`

- `Kalman Filter`

- State Space Model

- Change Point Detection(CPD)

- `Autoregressive conditional heteroskedasticity(ARCH)`

- [`Generalized Autoregressive Conditional Heteroskedasticity(GARCH)`](https://en.wikipedia.org/wiki/Autoregressive_conditional_heteroskedasticity)

#### 2) 기계학습/딥러닝 알고리즘

- `컴퓨팅` 기반 `인공지능 알고리즘`으로 `정확성 높은 비선형 관계` 추론

- `Prophet`

- `Neural Prophet`

- `RNN(Recurrent Neural Network)`

- `LSTM(Long Short-Term Memory)`

- `GRU(Gated Recurrent Unit)`

- Neural Networks Autoregression(NNAR)

- Attention

- Self-attention

- Transformer

#### 3) Platforms

- 글로벌 기업들이 독자적으로 개발한 `시계열 분석 플랫폼` 확대중

- [Amazon Forecast](https://aws.amazon.com/ko/forecast/)

- [Automated ML Time-series Forecasting at Microsoft Azure](https://azure.microsoft.com/en-us/blog/build-more-accurate-forecasts-with-new-capabilities-in-automated-machine-learning/)

- [Time Series Forecasting with Google Cloud AI Platform](https://codelabs.developers.google.com/codelabs/time-series-forecasting-with-cloud-ai-platform#0)

In [1]:
# Auto reload and user defined functions
%load_ext autoreload
%autoreload 2
from module_regression import *
from module_classification import *
from module_timeseries import *

# Time series algorithms
from statsmodels.tsa.arima_process import arma_generate_sample
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import AutoARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, plot_predict
from statsmodels.tsa.vector_ar.var_model import VAR, VARProcess
from statsmodels.tsa.vector_ar.util import varsim
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import coint

# **다변량 선형과정 알고리즘(Multivariate Linear Process)**

- **배경** : `과거`에는 주로 `단변량(Univariate)`의 시계열 데이터 `분석과 예측` 문제가 메인
`최근`에는 실시간으로 시차를 두고 상호 상관관계가 다양하게 이루어 지면서, `다변량(Multivariate)`의 시계열 데이터를 다뤄야 하는 경우가 `많아지고 있음`

    - `어떤 상품의 수요`($Y_{1,t}$)가 `다른 상품들의 수요`($Y_{2,t}, Y_{3,t}, ...$) 변화에 영향을 받거나

    - 어떤 지역의 `택시 수요`($Y_{1,t}$)가 `다른 지역의 택시 수요`($Y_{2,t-2}, Y_{3,t-1}, ...$)와 `일정한 시간 차이`을 두고 상관 관계를 보일 수 있음

    - 디지털시대에 접어들며, 변수가 적게는 `수백에서 많게는 수백만`에 이를만큼 `서로 상호작용이 다양`해지고 있고, 이를 효율적으로 처리 및 학습할 수 있는 알고리즘이 매우 중요

- **필요성**

    - 시간패턴을 반영하려는 `기계학습과 딥러닝` 알고리즘은 쉽게 `다변량 예측으로 확장`되어 활용

    - 단변량(Simple/Multiple) 시계열은 종속변수($Y_t$)가 `독립변수들에만! 영향을 받는다는 큰 가정` 존재

    - `현실`적으론 `종속변수와 독립변수는 상호 영향`을 주고받지 한쪽만 받는관계 아님

        - **예시:** 개인 소득과 지출 중 어떤게 `Y로 적합한가?`라는 질문은 왜 하지 않는가?

        - **예시:** 매출과 비용? 자전거수요와 날씨? 중 어떤게 `Y로 적합한가?`라는 질문은 왜 하지 않는가?

        - **예시:** 미국과 한국 주식시장 중 어떤게 `Y로 적합한가?`라는 질문은 왜 하지 않는가?

- **2차원(소득과 지출 모두를 종속변수) 과거1시점까지만을 고려하는 `백터자기회귀 알고리즘`**

\begin{align*}
\text{Equation of VAR(1)} && Y_{[t]} &= A Y_{[t-1]} + \epsilon_{[t]} \\
\text{where} && Y_{[t]} &= \begin{bmatrix} Y_{1t} \\ Y_{2t} \end{bmatrix},
A = \begin{bmatrix} \phi_{11} \phi_{12} \\ \phi_{21} \phi_{22} \end{bmatrix},
Y_{[t-1]} = \begin{bmatrix} Y_{1t-1} \\ Y_{2t-1} \end{bmatrix}, \\
&& \epsilon_{[t]} &= \begin{bmatrix} e_{1t} \\ e_{2t} \end{bmatrix} \sim N(0, \textstyle \sum_{\epsilon_{[t]}}) \\
\text{Each Equation of VAR(1)} && Y_{1t} &= \phi_{11} Y_{1t-1} + \phi_{12} Y_{2t-1} + e_{1t} \\
&& Y_{2t} &= \phi_{21} Y_{1t-1} + \phi_{22} Y_{2t-1} + e_{2t} \\
\end{align*}